# Product-level assets for Israeli ETNs

This notebook will download and save (in CSV format) product-level asset data reported by Israeli ETN issuers to the Tel Aviv Stock Exchange.  
  
The output CSV file contains:
 - A product identifier resembling a CUSIP value
 - Assets denominated in NIS 1000s (New Israeli Shekel in 1000s)
 - Assets denominated in US dollars
 - Assets denominated in US dollars (millions)

### 1. Imports and functions

In [1]:
import pandas as pd
from bs4 import BeautifulSoup as bs
import requests
from selenium import webdriver
import re
import time
from io import StringIO
from forex_python.converter import CurrencyRates
from datetime import datetime

In [2]:
def find_hrefs_by_regex(bs, reg_exp):
    all_items = bs.find_all("a", href=re.compile(reg_exp))
    return all_items

def convert_to_usd(val):
    val = float(val)
    usd_val = (val / fx_rate) * 1000
    rounded_usd_val = round(usd_val, 2)
    return rounded_usd_val

### 2. Set variables and initialize objects

Note that `date_from` and `date_to` below refer to the date report was submitted. Usually, the reports are submitted within the first week of the month.  

If, for example, you wish to retrieve end of April data, set `date_from` to May 1st, and `date_to` to around May 15th.
  
`date_month_end` is used primarily for obtaining the month-end USD/ILS exchange rate. Use a business day to ensure a value is returned. This date will also be stored in the CSV file name.

In [3]:
date_from = '2017-02-01'
date_to = '2017-02-15'
date_month_end = '2017-01-31'

public_exposure = 'ציבור וחשיפה'
cusip = 'מספר נייר ערך'
assets = 'שווי החזקות ציבור בהתאם למחיר בבורסה (באלפי שח)'

directory_to_save_csv = 'data'
csv_file_name = 'israel_product_data_{}.csv'.format(date_month_end)

*Note that when testing this script I noticed when setting the dates to December 2016 that one of the files had been mislabeled with the below title. If you find that the final product count / total assets for Israel is too low, this may be the cause.*  
  
*In this instance, I was able to uncomment the below line and amend code (currently commented out) further down in sections 3 and 4. However, doing this may break the script on other occasions.*

In [4]:
# public_holdings = 'החזקות ציבור במכשירים פיננסים'

This notebook is set up to operate using Chrome driver but can be easily customized for use with other browsers.

In [5]:
chrome_driver = "/Applications/chromedriver"

### 3. Collect report urls

In [6]:
url = 'http://maya.tase.co.il/reports/etn?q=%7B%22DateFrom%22:%22{}T21:00:00.000Z%22,%22DateTo%22:%22{}T22:00:00.000Z%22,%22QOpt%22:1,%22events%22:%5B2000%5D,%22subevents%22:%5B913%5D,%22Page%22:1%7D'.format(date_from, date_to)
url

'http://maya.tase.co.il/reports/etn?q=%7B%22DateFrom%22:%222017-02-01T21:00:00.000Z%22,%22DateTo%22:%222017-02-15T22:00:00.000Z%22,%22QOpt%22:1,%22events%22:%5B2000%5D,%22subevents%22:%5B913%5D,%22Page%22:1%7D'

In [7]:
driver = webdriver.Chrome(chrome_driver)
driver.get(url)
time.sleep(5)
html = bs(driver.page_source, 'lxml')
driver.quit()

In [8]:
divs = html.find_all('div', class_='feedItem ng-scope')

In [9]:
report_urls = []
for div in divs:
    report_url = find_hrefs_by_regex(div, 'reports*')
    title = report_url[0]['title']
    
    # Switch the below two lines if you apply the public_holdings fix
#     if public_exposure or public_holdings in title:
    if public_exposure in title:
        # full url is the path to the txt file of the data
        full_url = '{}{}{}'.format('http://maya.tase.co.il/', report_url[0]['href'],'/3/0')
        report_urls.append(full_url)

In [10]:
report_urls

['http://maya.tase.co.il/reports/details/1082126/3/0',
 'http://maya.tase.co.il/reports/details/1080385/3/0',
 'http://maya.tase.co.il/reports/details/1080372/3/0',
 'http://maya.tase.co.il/reports/details/1079941/3/0',
 'http://maya.tase.co.il/reports/details/1079804/3/0']

### 4. Visit each report url and store data in DataFrames

In [11]:
list_of_dfs = []

In [12]:
driver = webdriver.Chrome(chrome_driver)

for url in report_urls:
    
    # visit url and locate the text file
    driver.get(url)
    time.sleep(5)
    html = bs(driver.page_source, 'lxml')
    txt_file = html.find('button', class_='mob fullScreenButton').parent['href']
    t = requests.get(txt_file)
    t.encoding = 'UTF-8'
    
    # store text file in DataFrame
    data = pd.read_csv(StringIO(t.text), delimiter='\t')
    
    # find the relevant columns
    cols = {}
    count = 0
    for c in data.columns:
        count += 1
        if cusip in c:
            cols['cusip'] = count - 1
        if assets in c:
            cols['assets'] = count - 1
        # if you uncomment the public_holdings line, uncomment these two lines too
#         elif public_holdings in c:
#             cols['assets'] = count - 1
    
    # create new DataFrame containing only these two columns
    product_data = data.ix[:,[cols['cusip'],cols['assets']]]
    product_data.columns = ['cusip','assets_nis_1000s']
    product_data = product_data.dropna(axis=0)
    
    # get the USD-ILS exchange rate at month-end
    date_obj = datetime.strptime(date_to, "%Y-%m-%d")
    c = CurrencyRates()
    fx_rate = c.get_rate('USD', 'ILS', date_obj)
    
    # convert asset data into US dollars
    product_data['assets_usd'] = product_data['assets_nis_1000s'].apply(convert_to_usd)
    product_data['assets_usd_mn'] = round(product_data['assets_usd'] / 1000000, 2)
    
    # store DataFrame in list
    list_of_dfs.append(product_data)
    
driver.quit()

### 5. Concatenate DataFrames and drop duplicate values

Sometimes an issuer may upload a revised file, so to account for any duplicates the code removes any duplicate values and keeps the first value it finds (the website is ordered by date in descending order so newer data will be first).

In [13]:
len(list_of_dfs)

5

In [14]:
all_data = pd.concat(list_of_dfs)
len(all_data)

898

In [15]:
all_data.sample(5)

,cusip,assets_nis_1000s,assets_usd,assets_usd_mn
104,1133917.0,158094.0,4.220567e+07,42.21
88,1131291.0,315689.0,8.427812e+07,84.28
69,1135086.0,41979.0,1.120695e+07,11.21
96,1109370.0,648005.0,1.729951e+08,173.00
133,1138007.0,5234.0,1.397298e+06,1.40


In [16]:
all_data = all_data.drop_duplicates(subset='cusip', keep='first')
len(all_data)

690

In [17]:
total_israel_aum = all_data['assets_usd_mn'].sum()
total_israel_aum

29201.61999999997

In [18]:
number_of_products = len(all_data)
number_of_products

690

### 6. Export to CSV

In [19]:
all_data.to_csv('{}/{}'.format(directory_to_save_csv, csv_file_name), index=None)